In [1]:
import tensorflow as tf
tf.keras.mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070, compute capability 8.6


In [2]:
log_dir = "logs"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [3]:
import proteinbert
from proteinbert import load_pretrained_model
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
from Bio import SeqIO
import numpy


In [4]:
pretrained_model_generator, input_encoder = load_pretrained_model(local_model_dump_dir='proteinbert_modeldir')

In [5]:
train_path = "CAFA3_training_data/uniprot_sprot_exp.fasta"

In [6]:
MAX_SEQ_LEN = 1024

In [21]:
import numpy as np
import tqdm
def chunks(l, size):
    return [l[i:i+size] for i in range(0, len(l), size)]

def piecewise_predict(sequences_file, output_name, superbatch_size=500, batch_size=16):
    seqs = []
    ids = []
    for seq_record in SeqIO.parse(sequences_file, "fasta"):
        seqs.append(str(seq_record.seq))
        ids.append(str(seq_record.id))
    MAX_SEQ_LEN = 1024
    model = get_model_with_hidden_layers_as_outputs(pretrained_model_generator.create_model(MAX_SEQ_LEN))
    trimmed_seqs = [x[:MAX_SEQ_LEN-2] for x in seqs]
    all_ems = []
    for superbatch in tqdm.tqdm(chunks(trimmed_seqs, superbatch_size)):
        X = input_encoder.encode_X(superbatch, MAX_SEQ_LEN)
        _, superbatch_em = model.predict(X, batch_size=batch_size)
        all_ems.append(superbatch_em)
    concat_ems = np.concatenate(all_ems)
    ids_arr = np.asarray(ids)
    np.save(f"{output_name}_embeddings.npy", concat_ems)
    np.save(f"{output_name}_ids.npy", ids_arr)
    return concat_ems, ids_arr

In [28]:
ems, ids = piecewise_predict(train_path, "proteinbert_train", superbatch_size=512, batch_size=32)

  0%|          | 0/131 [00:00<?, ?it/s]

16/16 [==============================] - 2s 39ms/step


  1%|          | 1/131 [00:03<07:23,  3.41s/it]

16/16 [==============================] - 1s 38ms/step


  2%|▏         | 2/131 [00:05<05:12,  2.42s/it]

...

 98%|█████████▊| 129/131 [03:49<00:03,  1.77s/it]

16/16 [==============================] - 1s 39ms/step


 99%|█████████▉| 130/131 [03:51<00:01,  1.78s/it]

9/9 [==============================] - 2s 54ms/step


100%|██████████| 131/131 [03:54<00:00,  1.79s/it]


In [22]:
test_path = "cafa3_targets.fasta"

In [23]:
ems, ids = piecewise_predict(test_path, "proteinbert_test")

  0%|          | 0/7 [00:00<?, ?it/s]

32/32 [==============================] - 2s 26ms/step


 14%|█▍        | 1/7 [00:03<00:21,  3.59s/it]

32/32 [==============================] - 1s 26ms/step


 29%|██▊       | 2/7 [00:05<00:13,  2.62s/it]

...

 71%|███████▏  | 5/7 [00:11<00:04,  2.08s/it]

32/32 [==============================] - 1s 25ms/step


 86%|████████▌ | 6/7 [00:13<00:02,  2.04s/it]

21/21 [==============================] - 1s 26ms/step


100%|██████████| 7/7 [00:14<00:00,  2.09s/it]
